This goal of this script is to find GRBs that line up with the TESS mission in the GCN circulars archive.

The observation should have occurred between the start of TESS observation - which would be 2018-07-25 according to the Tess data release notes - up to the current date. 

Also all observations must be in the southern hemisphere, where sectors 1 - 13 are located.

Use swift data first, then other missions may be observed.

1. Set start and end dates for targets being observed, in this case start is GRB180725 (if it exists).
2. Find GRBs associated with swift observations.
3. Open circular file. (Memory Mapping, load file into memory and read matching strings from there.)
4. Find RA and DEC coordinates, filter out positive DEC coordinates.
5. Return link, coordinates, and identifier.
6. Run RA and DEC through tesscut to determine which sectors observation overlaps with
7. Determine date difference between sector observation date and swift observation date.

In [78]:
import numpy as np
import astropy as ap
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import sys
from datetime import datetime
from dateutil.parser import parse
from itertools import chain

In [93]:
sys.setrecursionlimit(10000)

# Following takes NASA circular webpage and turns it 
# into a Beautiful Soup object that can be used to match certain requirements

circ_Arch = None  
circ_Arch = requests.get('https://gcn.gsfc.nasa.gov/gcn/gcn3_archive.html')
circ_old1 = requests.get('https://gcn.gsfc.nasa.gov/gcn3_arch_old142.html')
circ_old2 = requests.get('https://gcn.gsfc.nasa.gov/gcn3_arch_old141.html')

pages = []
pages.append(circ_Arch)
pages.append(circ_old1)
pages.append(circ_old2)

#circ_soup is a list of gcn archives as beautiful soup objects
circ_soup = []
i = 0
for page in pages:
    circ_soup.append(BeautifulSoup(page.text, 'html.parser'))
    circ_soup[i].prettify
    i += 1

# Regular Expressions for categorizing the elements in the circular list
# id_re returns the GRB and its number.
# obs_desc returns the description of the observation
# sn_re returns the serial numbers of each circular.

swift_re = re.compile(r'.*Swift detection.*')
id_re = re.compile(r'\D\D\D \d\d\d\d\d\d\D:')
obs_desc = re.compile(r'Swift.*')
sn_re = re.compile(r'(\d\d\d\d\d)')

#Creates a list to hold all strings that match the swift regular expression.
archive_obs = []
# ul_array holds all of the html elements in each archive.
# Useful for navigating through desired swift ciculars, and extracting desired tags.
ul_array = []

for archive in circ_soup:
    # The circulars are sectioned in an unordered list.
    circ_list = archive.find('ul')
    #Each element in the unordered list contains a circular link, an identifier and what made the observation.
    # Circ_text is each li element in a text format.
    circ_text = circ_list.find_next('li').get_text()
    swift_obs.append(swift_re.findall(circ_text))
    ul_array.append(swift_re.findall(str(circ_list)))

#flatten the swift observation list
swift_flat = list(chain.from_iterable(swift_obs))
ul_flat = list(chain.from_iterable(ul_array))
print(ul_flat)

['<li><a href="gcn3/27254.gcn3">27254</a> GRB 200228B: Swift detection of a burst<br/>', '<li><a href="gcn3/27234.gcn3">27234</a> GRB 200227A: Swift detection of a burst<br/>', '<li><a href="gcn3/27173.gcn3">27173</a> GRB 200224A: Swift detection of a burst<br/>', '<li><a href="gcn3/27125.gcn3">27125</a> GRB 200219A: Swift detection of a short burst<br/>', '<li><a href="gcn3/27102.gcn3">27102</a> GRB 200216B: Swift detection of a burst<br/>', '<li><a href="gcn3/27079.gcn3">27079</a> GRB 200215A: Swift detection of a burst<br/>', '<li><a href="gcn3/26993.gcn3">26993</a> GRB 200205B: Swift detection of a burst with optical counterpart<br/>', '<li><a href="gcn3/26982.gcn3">26982</a> GRB 200205A: Swift detection of a burst with a possible UV counterpart<br/>', '<li><a href="gcn3/26953.gcn3">26953</a> GRB 200131A: Swift detection of a burst with an optical counterpart<br/>', '<li><a href="gcn3/26895.gcn3">26895</a> GRB 200127A: Swift detection of a burst<br/>', '<li><a href="gcn3/26876.gcn3

In [94]:
# This cell produced a preliminary table. The proper data set is in the following cells

# This for loop creates a list of the grb identifiers that fall under the swift observations.
i = 0
grb_ids = []
desc_list = []
sn_list = []

#element is one line in the swift obs that contains the circular #, identifier and observation description
for element in swift_flat:
    grb_ID = id_re.findall(element)
    desc = obs_desc.findall(element)
    sn = sn_re.findall(element)
    # Some empty lists are in the original ul tag so these are skipped
    if len(grb_ID) == 0:
        continue
    else:
        grb_ids.append(grb_ID[0]) #Using index notation to append the string element instead of a list object.
        desc_list.append(desc[0])
        sn_list.append(sn[0])
print(len(sn_list))

960


In [97]:

# Regular expression for all href link names
gcn_re = re.compile(r'gcn3.*gcn3')

#Following code filters out the circular links from the tags, and puts them into a list.
href_list = []
for li in ul_flat:
    gcn = gcn_re.findall(li)
    href_list.append(gcn[0])

# Every circular link has the following string preceding its circular number.
link = "https://gcn.gsfc.nasa.gov/"

#Each string in href_list is appended to link to create the full gcn link for the corresponding observation.
# These are then appended to link_list.
link_list = []
for circular in href_list:
    link_gcn = link + circular #Main link + corresponding circular string
    link_list.append(link_gcn)

In [98]:
gcn_text = []
#Each complete circular address in link_list is requested and converted into string format.
# These strings are then appended to a list, so that each circular can be parsed through
# to find coordinates and dates.
for gcn in link_list:
    link_src = requests.get(gcn)
    gcn_circ = link_src.text
    gcn_text.append(gcn_circ)

In [99]:
#Regular expressions for date, RA, DEC and identifiers
date_re = re.compile(r'\d\d.*GMT')
cord_re = re.compile(r'RA, Dec.*\d\d\d')


#Lists for each category
date_list = []
cord_list = []
id_list = []

for text in gcn_text:
    date = date_re.findall(text)
    cord = cord_re.findall(text)
    i_d = id_re.findall(text)
    
    # If else statement to overlook any empty lists, which will skip over duplicate identifiers.
    if len(i_d) == 0:
        continue
    else:
        date_list.append(date[0])
        cord_list.append(cord)
        id_list.append(i_d)
         
print(len(cord_list))
print(len(date_list))
print(len(id_list))

120
120
120


In [100]:
# Creates a table from the lists made in the previous cell.
pd.DataFrame(list(zip(id_list, cord_list, date_list, link_list)), columns= ["ID", "RA, Dec", "Date", "Circular"])

,ID,"RA, Dec",Date,Circular
0,[GRB 200228B:],"[RA, Dec 252.026, +16.960, RA, Dec 252.04730, ...",20/02/28 11:27:30 GMT,https://gcn.gsfc.nasa.gov/gcn3/27254.gcn3
1,[GRB 200227A:],"[RA, Dec 56.436, +9.491, RA, Dec 56.43302, 9.4...",20/02/27 07:37:50 GMT,https://gcn.gsfc.nasa.gov/gcn3/27234.gcn3
2,[GRB 200224A:],"[RA, Dec 248.794, +41.675, RA, Dec 248.7668, 4...",20/02/24 03:41:23 GMT,https://gcn.gsfc.nasa.gov/gcn3/27173.gcn3
3,[GRB 200219A:],"[RA, Dec 342.644, -59.108]",20/02/19 07:55:10 GMT,https://gcn.gsfc.nasa.gov/gcn3/27125.gcn3
4,[GRB 200216B:],"[RA, Dec 160.461, +19.454]",20/02/16 13:57:44 GMT,https://gcn.gsfc.nasa.gov/gcn3/27102.gcn3
5,[GRB 200215A:],"[RA, Dec 34.122, +12.781, RA, Dec 34.0794]",20/02/15 15:14:13 GMT,https://gcn.gsfc.nasa.gov/gcn3/27079.gcn3
6,[GRB 200205B:],"[RA, Dec 107.861, -56.506, RA, Dec 107.7881]",20/02/05 19:46:01 GMT,https://gcn.gsfc.nasa.gov/gcn3/26993.gcn3
7,[GRB 200205A:],"[RA, Dec 130.171, -35.245, RA, Dec 130.1717, -...",20/02/05 07:14:19 GMT,https://gcn.gsfc.nasa.gov/gcn3/26982.gcn3
8,[GRB 200131A:],"[RA, Dec 3.095, +51.124, RA, Dec 3.0942, 51.1158]",20/01/31 22:53:30 GMT,https://gcn.gsfc.nasa.gov/gcn3/26953.gcn3
9,[GRB 200127A:],"[RA, Dec 295.204, +44.910, RA, Dec 295.19447]",20/01/27 15:04:17 GMT,https://gcn.gsfc.nasa.gov/gcn3/26895.gcn3


In [101]:
# Regular expression for parsing coordinates and their labels.
ra_exp = re.compile('RA')
dec_exp = re.compile('Dec')
cord_exp = re.compile(r'\-?\d?\d?\d.\d\d\d')

ra_list = []
dec_list = []

for cord in cord_list:
    ra_cord = cord_exp.findall(cord[0])[0] # First match in each coordinate element is ra
    dec_cord = cord_exp.findall(cord[0])[1]# Second match in each coordinate element is dec
    
    ra_list.append(ra_cord)
    dec_list.append(dec_cord)


In [104]:
# Results from tesscut api are appended to the following lists.
sec_list = []
sec_names = []
sec_cams = []
ccd_list = []


for i in range(0,120):
    payload = {'ra': ra_list[i], 'dec': dec_list[i], 'radius':'1m'}
    r = requests.get('https://mast.stsci.edu/tesscut/api/v0.1/sector', params = payload)
    
    #Result is a list and returns the values generated by the tesscut api.
    result = r.json()['results']
    
    # Certain coordinates do not return any sector results, filtered out and
    # A N/A value is returned to a list using an if else statement.
    
    if len(result) == 0:
        sec_list.append('N/A')
        sec_names.append('N/A')
        sec_cams.append('N/A')
        ccd_list.append('N/A')
    elif len(result) == 1:
        sec_list.append(result[0]['sector'])
        sec_names.append(result[0]['sectorName'])
        sec_cams.append(result[0]['camera'])
        ccd_list.append(result[0]['ccd'])
    else:
        sec_list.append(result[0]['sector'] + " " + result[1]['sector'])
        sec_names.append(result[0]['sectorName'] + " " + result[1]['sectorName'])
        sec_cams.append(result[0]['camera'] + " " + result[1]['camera'])
        ccd_list.append(result[0]['ccd'] + " " + result[1]['ccd'])

In [105]:
big_table = pd.DataFrame(list(zip(id_list, cord_list, date_list, link_list,sec_list, sec_names, sec_cams, ccd_list)), 
             columns= ["ID", "RA, Dec", "Swift Date", "Circular", "Sector", "SectorName", "Camera", "ccd"])
big_table

,ID,"RA, Dec",Swift Date,Circular,Sector,SectorName,Camera,ccd
0,[GRB 200228B:],"[RA, Dec 252.026, +16.960, RA, Dec 252.04730, ...",20/02/28 11:27:30 GMT,https://gcn.gsfc.nasa.gov/gcn3/27254.gcn3,N/A,N/A,N/A,N/A
1,[GRB 200227A:],"[RA, Dec 56.436, +9.491, RA, Dec 56.43302, 9.4...",20/02/27 07:37:50 GMT,https://gcn.gsfc.nasa.gov/gcn3/27234.gcn3,0005,tess-s0005-1-4,1,4
2,[GRB 200224A:],"[RA, Dec 248.794, +41.675, RA, Dec 248.7668, 4...",20/02/24 03:41:23 GMT,https://gcn.gsfc.nasa.gov/gcn3/27173.gcn3,N/A,N/A,N/A,N/A
3,[GRB 200219A:],"[RA, Dec 342.644, -59.108]",20/02/19 07:55:10 GMT,https://gcn.gsfc.nasa.gov/gcn3/27125.gcn3,0001,tess-s0001-2-1,2,1
4,[GRB 200216B:],"[RA, Dec 160.461, +19.454]",20/02/16 13:57:44 GMT,https://gcn.gsfc.nasa.gov/gcn3/27102.gcn3,N/A,N/A,N/A,N/A
5,[GRB 200215A:],"[RA, Dec 34.122, +12.781, RA, Dec 34.0794]",20/02/15 15:14:13 GMT,https://gcn.gsfc.nasa.gov/gcn3/27079.gcn3,N/A,N/A,N/A,N/A
6,[GRB 200205B:],"[RA, Dec 107.861, -56.506, RA, Dec 107.7881]",20/02/05 19:46:01 GMT,https://gcn.gsfc.nasa.gov/gcn3/26993.gcn3,0002 0003,tess-s0002-4-4 tess-s0003-4-4,4 4,4 4
7,[GRB 200205A:],"[RA, Dec 130.171, -35.245, RA, Dec 130.1717, -...",20/02/05 07:14:19 GMT,https://gcn.gsfc.nasa.gov/gcn3/26982.gcn3,0008,tess-s0008-2-2,2,2
8,[GRB 200131A:],"[RA, Dec 3.095, +51.124, RA, Dec 3.0942, 51.1158]",20/01/31 22:53:30 GMT,https://gcn.gsfc.nasa.gov/gcn3/26953.gcn3,0017,tess-s0017-2-1,2,1
9,[GRB 200127A:],"[RA, Dec 295.204, +44.910, RA, Dec 295.19447]",20/01/27 15:04:17 GMT,https://gcn.gsfc.nasa.gov/gcn3/26895.gcn3,0014 0015,tess-s0014-2-4 tess-s0015-2-3,2 2,4 3


In [106]:
# The list of sectors and their start and end dates are stored in the dictionary sec_date.
sec_date = {"Sec26" : ["2020-06-08", "2020-07-04"], "Sec25" : ["2020-05-13", "2020-06-08"], 
            "Sec24" : ["2020-04-16", "2020-05-13"], "Sec23" : ["2020-03-18", "2020-04-16"], 
            "Sec22" : ["2020-02-18", "2020-03-18"], "Sec21" : ["2020-01-21", "2020-02-18"], 
            "Sec20" : ["2019-12-24", "2020-01-21"], "Sec19" : ["2019-11-27", "2019-12-24"], 
            "Sec18" : ["2019-11-02", "2019-11-27"], "Sec17" : ["2019-10-07", "2019-11-02"], 
            "Sec16" : ["2019-09-11", "2019-10-07"], "Sec15" : ["2019-08-15", "2019-09-11"], 
            "Sec14" : ["2019-07-18", "2019-08-15"], "Sec13" : ["2019-06-19", "2019-07-18"], 
            "Sec12" : ["2019-05-21", "2019-06-19"], "Sec11" : ["2019-04-22", "2019-05-21"], 
            "Sec10" : ["2019-03-26", "2019-04-22"], "Sec9" : ["2019-02-28", "2019-03-25"], 
            "Sec8" : ["2019-02-02", "2019-02-27"], "Sec7" : ["2019-01-08", "2019-02-01"], 
            "Sec6" : ["2018-12-12", "2019-01-06"], "Sec5" : ["2018-11-15", "2018-12-11"], 
            "Sec4" : ["2018-10-19", "2018-11-14"], "Sec3" : ["2018-09-20", "2018-10-17"], 
            "Sec2" : ["2018-08-23", "2018-09-20"], "Sec1" : ["2018-07-25", "2018-08-22"]}

#Grabs the dates column from the big table.https://archive.stsci.edu/tess/tess_drn.html
swift_arr = big_table['Swift Date']

#List of tess sector dates as datetime objects.
# Each sector is in a list, containing its beg and end dates.
tess_dt = []

#Converts sec_date values to datetime objects
for dates in sorted(sec_date.values()):
    dates[0] = (datetime.strptime(dates[0], "%Y-%m-%d")) #sector beginning date
    dates[1] = (datetime.strptime(dates[1], "%Y-%m-%d")) #sector end date  
    tess_dt.append(dates)

# Loop chops off timezone for conversion to datetime object.
# Converted to new list and appended to swift_dt list.
swift_dt = []
for swift_date in swift_arr:
    swift_date = (datetime.strptime(swift_date[0:8], "%y/%m/%d"))
    swift_dt.append(swift_date)
print(tess_dt)

[[datetime.datetime(2018, 7, 25, 0, 0), datetime.datetime(2018, 8, 22, 0, 0)], [datetime.datetime(2018, 8, 23, 0, 0), datetime.datetime(2018, 9, 20, 0, 0)], [datetime.datetime(2018, 9, 20, 0, 0), datetime.datetime(2018, 10, 17, 0, 0)], [datetime.datetime(2018, 10, 19, 0, 0), datetime.datetime(2018, 11, 14, 0, 0)], [datetime.datetime(2018, 11, 15, 0, 0), datetime.datetime(2018, 12, 11, 0, 0)], [datetime.datetime(2018, 12, 12, 0, 0), datetime.datetime(2019, 1, 6, 0, 0)], [datetime.datetime(2019, 1, 8, 0, 0), datetime.datetime(2019, 2, 1, 0, 0)], [datetime.datetime(2019, 2, 2, 0, 0), datetime.datetime(2019, 2, 27, 0, 0)], [datetime.datetime(2019, 2, 28, 0, 0), datetime.datetime(2019, 3, 25, 0, 0)], [datetime.datetime(2019, 3, 26, 0, 0), datetime.datetime(2019, 4, 22, 0, 0)], [datetime.datetime(2019, 4, 22, 0, 0), datetime.datetime(2019, 5, 21, 0, 0)], [datetime.datetime(2019, 5, 21, 0, 0), datetime.datetime(2019, 6, 19, 0, 0)], [datetime.datetime(2019, 6, 19, 0, 0), datetime.datetime(2019

In [107]:
# List grabs sectors from big_table - related sectors 
sectors = big_table['Sector']
# Convert sectors to integers, used to iterate through tess_dt dictionary by sector.
sec_ints = []
for sec in sectors:
    if sec[0] == '0':
        sec = int(sec[:4])
        sec_ints.append(sec)
    else:
        sec = sec[:4]
        sec_ints.append(sec)

# Iterate through the dates of the corresponding sectors of tess.
# Used to compare tess dates with swift dates.
rel_sec = []
for sec_num in sec_ints:
    if type(sec_num) == int:
        rel_sec.append(tess_dt[sec_num - 1])
        
    else:
        rel_sec.append("N/A")

# Swift count is used to iterate through swift_dt simultaneously with the rel_sec loop
# This is so the differences in dates can be appended to a list.
swift_count = 0

# List of tess start and end date differences
strt_diffs = []
end_diffs = []

for date in rel_sec:
    if len(date) == 2:
        start_diff = swift_dt[swift_count] - date[0] # Difference between swift observation and tess sector beg date
        end_diff = swift_dt[swift_count] - date[1] # "                                            " end date
        strt_diffs.append(start_diff)
        end_diffs.append(end_diff)
    else:
        strt_diffs.append("N/A")
        end_diffs.append("N/A")
    swift_count += 1
    
print(len(strt_diffs))

120


In [108]:
bigger_table = pd.DataFrame(list(zip(id_list, cord_list, date_list, link_list,sec_list, sec_names, sec_cams, ccd_list,
                                    strt_diffs, end_diffs)), 
             columns= ["ID", "RA, Dec", "Swift Date", "Circular", "Sector", "SectorName", "Camera", "ccd", 
                       "Difference between TESS sector start date and Swift Obs Date", 
                       "Difference between TESS sector end date and Swift Obs Date"])
bigger_table.to_html("GRB_Table.html")